In [20]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Activation
from keras.utils import to_categorical

In [2]:
IMG_SIZE = (28, 28)
TRAIN_N = 42000
TEST_N = 28000

In [30]:
def load(filename, N, load_label=False):
  imgs = np.empty((N, IMG_SIZE[0] * IMG_SIZE[1]), dtype=np.float32)
  labels = np.empty((N), dtype=np.int32)
  with open(filename, 'r') as f:
    for i, line in enumerate(f):
      if i == 0:
        continue
      for j, value in enumerate(line.split(',')):
        if load_label:
          if j == 0:
            labels[i - 1] = int(value)
          else:
            imgs[i - 1][j - 1] = float(value)
        else:
          imgs[i - 1][j] = float(value)
  imgs = (imgs / 127.5) - 1.
  imgs = imgs.reshape((N, IMG_SIZE[0], IMG_SIZE[1], 1))
  if load_label:
    labels = to_categorical(labels, num_classes=10)
    return imgs, labels
  return imgs

In [31]:
train_imgs, train_labels = load('train.csv', TRAIN_N, load_label=True)

In [32]:
test_imgs = load('test.csv', TEST_N)

In [34]:
model = Sequential()
model.add(Conv2D(20, (2, 2), strides=(2, 2), padding='same', activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 1)))
model.add(Conv2D(50, (2, 2), strides=(2, 2), padding='same', activation='relu'))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [35]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_imgs, train_labels, epochs=5, batch_size=32)

Epoch 1/5
42000/42000 [==============================] - 10s 250us/step - loss: 0.2118 - acc: 0.9359
Epoch 2/5
42000/42000 [==============================] - 9s 226us/step - loss: 0.0666 - acc: 0.9787
Epoch 3/5
42000/42000 [==============================] - 9s 222us/step - loss: 0.0376 - acc: 0.9885
Epoch 4/5
42000/42000 [==============================] - 9s 225us/step - loss: 0.0248 - acc: 0.9919
Epoch 5/5
42000/42000 [==============================] - 9s 224us/step - loss: 0.0161 - acc: 0.9945


In [39]:
classes = model.predict(test_imgs, batch_size=32)

In [40]:
top_labels = np.argmax(classes, axis=1)

In [42]:
with open('submission.csv', 'w') as f:
  f.write('ImageId,Label\n')
  for i in range(top_labels.size):
    f.write('{},{}\n'.format(i + 1, top_labels[i]))